In [1]:
from layers import *
import tensorflow as tf

In [2]:
import tensorflow as tf
from tensorflow.keras import layers,activations,backend,constraints,initializers,regularizers

class NFM(layers.Layer):
    def __init__(self,
                 dropout_rate=0.2,
                 **kwargs):
        super().__init__(**kwargs)

    def call(self,inputs):
        pass

2023-10-08 10:09:01.043586: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-10-08 10:09:01.043752: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-08 10:09:01.044784: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
import itertools

import tensorflow as tf
from tensorflow.keras import layers

class DynamicMultiRNN(layers.Layer):
    def __init__(self, num_units=None, rnn_type='LSTM', return_sequence=True, num_layers=2, num_residual_layers=1, dropout_rate=0.2,
                 forget_bias=1.0, **kwargs):

        self.num_units = num_units
        self.return_sequence = return_sequence
        self.rnn_type = rnn_type
        self.num_layers = num_layers
        self.num_residual_layers = num_residual_layers
        self.dropout = dropout_rate
        self.forget_bias = forget_bias
        super(DynamicMultiRNN, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        input_seq_shape = input_shape[0]
        if self.num_units is None:
            self.num_units = input_seq_shape.as_list()[-1]
        if self.rnn_type == "LSTM":
            single_cell = tf.nn.rnn_cell.BasicLSTMCell(self.num_units, forget_bias=self.forget_bias)
        elif self.rnn_type == "GRU":
            single_cell = tf.nn.rnn_cell.GRUCell(self.num_units)
        else:
            raise ValueError("Unknown unit type %s!" % self.rnn_type)
        dropout = self.dropout if tf.keras.backend.learning_phase() == 1 else 0
        single_cell = tf.nn.rnn_cell.DropoutWrapper(cell=single_cell, input_keep_prob=(1.0 - dropout))
        cell_list = []
        for i in range(self.num_layers):
            residual = (i >= self.num_layers - self.num_residual_layers)
            if residual:
                single_cell_residual = tf.nn.rnn_cell.ResidualWrapper(single_cell)
                cell_list.append(single_cell_residual)
            else:
                cell_list.append(single_cell)
        if len(cell_list) == 1:
            self.final_cell = cell_list[0]
        else:
            self.final_cell = tf.nn.rnn_cell.MultiRNNCell(cell_list)
        super(DynamicMultiRNN, self).build(input_shape)

    def call(self, input_list, mask=None, training=None):
        rnn_input, sequence_length = input_list
        with tf.name_scope("rnn"), tf.variable_scope("rnn", reuse=tf.AUTO_REUSE):
            rnn_output, hidden_state = tf.nn.dynamic_rnn(self.final_cell, inputs=rnn_input, sequence_length=tf.squeeze(sequence_length),
                                                         dtype=tf.float32, scope=self.name)
        if self.return_sequence:
            return rnn_output
        else:
            return tf.expand_dims(hidden_state, axis=1)

    def compute_output_shape(self, input_shape):
        rnn_input_shape = input_shape[0]
        if self.return_sequence:
            return rnn_input_shape
        else:
            return (None, 1, rnn_input_shape[2])

    def get_config(self, ):
        config = {'num_units': self.num_units, 'rnn_type': self.rnn_type, 'return_sequence':self.return_sequence, 'num_layers': self.num_layers,
                  'num_residual_layers': self.num_residual_layers, 'dropout_rate': self.dropout}
        base_config = super(DynamicMultiRNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [38]:
digits = tf.ragged.constant([[1., 1., 1.],[2.],[3., 3., 3., 3.],[4., 4.]])
padded = digits.to_tensor(0.,shape=(3,2))
final_tensor = tf.reverse(padded, [-1])
final_tensor

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1., 1.],
       [0., 2.],
       [3., 3.]], dtype=float32)>

In [61]:
import tensorflow as tf
import numpy as np
test=tf.feature_column.sequence_categorical_column_with_identity("test",num_buckets=100)
testemb=tf.feature_column.embedding_column(test,dimension=16)

tmp=np.ones((8,4),dtype=np.int64)
tmp[:,2:]=-1
tmp=tf.ragged.constant([[1, 1, 1],[2],[3, 3, 3, 3],[4, 4]])
features={
    "test":tmp
}
inputs=tf.keras.experimental.SequenceFeatures([testemb])(features)

ValueError: TypeError: object of type 'RaggedTensor' has no len()


In [59]:
inputs[1]


<tf.Tensor: shape=(8,), dtype=int64, numpy=array([2, 2, 2, 2, 2, 2, 2, 2])>